# Sulcal morphology pipeline

We want to measure morphological properties of small and variable sulci. To do so, we'll need to  
(1) process the raw T1 scan in FreeSurfer,  
(2) manually identify and label the sulci, and  
(3) use FreeSurfer to extract anatomical properties from our sulcal labels.

The full pipeline is outlined below:

<img src="https://raw.githubusercontent.com/smaboudian/sulcal-morph-demo/main/images/sulcmorph-pipeline.png" width="900" />


The first step is already completed (because it takes a few hours per scan): running the _recon-all_ command in FreeSurfer. 

FreeSurfer (FS) is an open-source software package for neuroimaging (structural MRI) analysis and visualization<sup>[1]</sup>. [_Recon-all_](https://surfer.nmr.mgh.harvard.edu/fswiki/recon-all) performs the FS cortical reconstruction process, which converts the 3D cortical volume into 2D surfaces that it uses to compute various anatomical properties (that we'll use in step 3)<sup>[2]</sup>. Here's an overview of the steps:


<img src="https://raw.githubusercontent.com/smaboudian/sulcal-morph-demo/main/images/FS_reconall.png" width="800" /> 

<img src="https://raw.githubusercontent.com/smaboudian/sulcal-morph-demo/main/images/FS_reconall-output.png" width="800" />

We'll be labeling sulci on the *inflated* surface, because the inflation allows us to see the sulci (in red). We use the *pial* (the one that looks like a regular brain) as a reference, because sometimes the inflated one distorts the way the sulci look.


# Labeling sulci

The next step is to manually identify and label the sulci. We first do this on screenshots of each hemisphere and then enter finalized labels into FS to extract morphological properties from each label.

For this demo, we'll be labeling sulci in medial parietal cortex (MPC). In this region, there are 7 primary/secondary sulci and 1-5 tertiary sulci:


<img src="https://raw.githubusercontent.com/smaboudian/sulcal-morph-demo/main/images/labels_inflated_pial.png" width="1000" />


There is considerable individual variability in the number and location of sulci: there are 8 sulci that will be present in all hemispheres ("consistent"), and 4 variable or "inconsistent" sulci that not every hemisphere will have:

<img src="https://raw.githubusercontent.com/smaboudian/sulcal-morph-demo/main/images/MPC_sulci.png" width="1000" />



Head to [this link](https://docs.google.com/presentation/d/1bP4_0ZPlcH8Zn5mYyScCcsP61hnBiWtJg36Zq9l0v6k/edit?usp=sharing) to see some examples of labeled hemispheres and atlases we use for reference. Pick a few slides each to label (directly on the slide).


Once we've labeled the sulci on the screenshots and confirmed they're correct, we can enter them into FS.  
For this step you need to log into the Neuro Cluster; talk to Samira.   
*[probably will run through the process once myself in tksurfer to show them (like I do w/ RAs), & then will log them into the cluster in 2 groups on my laptop + ethans since we already have X2Go setup]*

Instructions for entering sulcal labels into FS are [here](https://sites.google.com/view/cnl-wiki/home/how-tos/freesufer/labels-in-freesurfer?authuser=0) (skip to step 5). 

# Extracting anatomical properties from sulcal labels & analyzing them

Running the [_mris_anatomical_stats_](https://surfer.nmr.mgh.harvard.edu/fswiki/mris_anatomical_stats) function in FS computes anatomical properties (surface area, gray matter volume, cortical thickness, etc.) for a set of labels (if specified), in this case the sulcal labels you just created. This can take some time, so you can skip this step for this demo. 

Now you can view the final output that contains the morphological metrics for the sulci you labeled. Run the code cells below to load in the data -- some basic plots and analyses are provided, but feel free to play around with it on your own too!

*[I kept this very simple but lmk if you think we should add more. data is from ADNI OA CNs, before we had x/y labels]*




In [ ]:
## load packages ##
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# load in your data: morphological metrics for your sulcal labels
url = 'https://raw.githubusercontent.com/smaboudian/sulcal-morph-demo/main/demo-30CN_MPC.csv'
df = pd.read_csv(url, converters={'item': eval})
df.head()

In [ ]:
# function to make violin plots, using Seaborn (just run this cell)

def violinplot(df, metric, metric_name):
    sulci = ['pos', 'prculs', 'prcus-p', 'prcus-i', 'prcus-a', 
             'spls', 'mcgs','sspls', 'ifrms', 'icgs-p']

    fig, axes = plt.subplots(1, 2, sharey=True, figsize=(15, 4))
    fig.suptitle(metric_name+' by sulcal type and hemisphere');

    axes[0].set_title('left hem.')
    sns.violinplot(ax=axes[0],
                   x="label", y=metric, 
                   hue="sulc_type", order=sulci,
                   data=df[df['hemi']=='lh']);
    axes[1].set_title('right hem.')
    axes[1].set_ylabel('');
    sns.violinplot(ax=axes[1],
                   x="label", y=metric, 
                   hue="sulc_type", order=sulci,
                   data=df[df['hemi']=='rh']);


In [ ]:
# plot mean sulcal cortical thickness (in mm)
violinplot(df, 'cortical_thickness_mean', 'sulcal cortical thickness')

In [ ]:
# plot mean sulcal depth (in mm)
violinplot(df, 'sulcal_depth_mm', 'sulcal depth')

It looks like tertiary sulci are both thicker and less deep than primary & secondary sulci. Let's confirm that:

In [ ]:
## setup rpy2 ##
# run once; this will allow you to run R embedded in your Python notebook
%load_ext rpy2.ipython


In [ ]:
## import robjects ##
import tzlocal
import rpy2.robjects 
from rpy2.robjects import r

In [ ]:
#### import rpackages ####

# import rpy2's package module
import rpy2.robjects.packages as rpackages

# import R's utility package
utils = rpackages.importr('utils')

# select a mirror for R packages
utils.chooseCRANmirror(ind=1) # select the first mirror in the list

In [ ]:
%%R 
#^ runs R in this cell

# load packages
library(nlme)

# load data in R
url = 'https://raw.githubusercontent.com/smaboudian/sulcal-morph-demo/main/demo-30CN_MPC.csv'
df = read.csv(url)

# run models
model_CT <- lme(cortical_thickness_mean ~ hemi * sulc_type, random = ~ 1|sub/hemi/sulc_type,
               data = df)
model_CT.aov <- anova(model_CT)
cat('Cortical Thickness ANOVA:\n')
print(model_CT.aov)

model_SD <- lme(sulcal_depth_mm ~ hemi * sulc_type, random = ~ 1|sub/hemi/sulc_type,
               data = df)
model_SD.aov <- anova(model_SD)
cat('\nSulcal Depth ANOVA:\n')
print(model_SD.aov)

In [ ]:
# feel free to run your own analyses in the cells below!

# Resources:

1. FreeSurfer wiki: https://surfer.nmr.mgh.harvard.edu/fswiki
2. FreeSurfer tutorial slides used in this demo: https://surfer.nmr.mgh.harvard.edu/fswiki/Tutorials 
3. FreeSurfer tutorial: https://andysbrainbook.readthedocs.io/en/latest/FreeSurfer/FreeSurfer_Introduction.html

